In [4]:
#imports
import os
from owlready2 import get_ontology
from owlready2 import sync_reasoner_hermit
import traceback
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

In [5]:
# Define the file path
directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
filename = 'oec-extracted.owl'
file_path = os.path.join(directory, filename)

try:
    # Load the ontology
    onto = get_ontology("file://" + file_path).load()
    print("Ontology loaded successfully!")

except Exception as e:
    # Print an error message if loading fails
    print("Error occurred during ontology loading:")
    print(e)

#check inconsistensies
inconsistencies = list(onto.inconsistent_classes())
if inconsistencies:
    print("Ontology is inconsistent!")
    print("Inconsistencies:")
    for inconsistency in inconsistencies:
        print(f"Inconsistency found involving: {inconsistency}")
else:
    print("Ontology is consistent.")    

Ontology loaded successfully!
Ontology is consistent.


In [6]:
#sync reasoner
try:
    with onto:
        sync_reasoner_hermit()
except Exception as e:
    print(f"Error: {e}")


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit;c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/Matheus/AppData/Local/Temp/tmplwdj1ls4
* Owlready2 * HermiT took 0.5283694267272949 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [7]:
# Get inferred object properties and their usage
inferred_properties = set()

for prop in onto.object_properties():
    try:
        for subj, obj in prop.get_relations():
            inferred_properties.add((prop, subj, obj))
    except Exception as e:
        print(f"An error occurred while processing object property '{prop.name}': {e}")

# Print inferred object properties and their usage
for prop, subj, obj in inferred_properties:
    try:
        subj_type = "Class" if subj in onto.classes() else "Individual" if subj in onto.individuals() else "Unknown"
        obj_type = "Class" if obj in onto.classes() else "Individual" if obj in onto.individuals() else "Unknown"

        print(f"Object Property '{prop.name}' inferred on {obj_type.lower()} '{obj.name}' because of '{subj.name}'")
    except Exception as e:
        print(f"An error occurred while formatting output: {e}")

Object Property 'contextDefinedByActorType' inferred on individual 'ContextDERHousehold' because of 'ActorDERHousehold'
Object Property 'exchangedDataHasTerm' inferred on individual 'ExchangedDataHouseholdToCompany' because of 'ExchangedTermNetMetering'
Object Property 'exchangedDataHasSender' inferred on individual 'ActorDERHousehold' because of 'ExchangedDataHouseholdToCompany'
Object Property 'termHasMeaningByContext' inferred on individual 'ContextUnknown' because of 'TermNetGridConnectionDERHousehold'
Object Property 'termHasWrittenContent' inferred on individual 'ExchangedTermStorage' because of 'TermStorageMeaningDERHousehold'
Object Property 'exchangedDataHasReceiver' inferred on individual 'ActorEnergyCompany' because of 'ExchangedDataHouseholdToCompany'
Object Property 'exchangedDataHasTerm' inferred on individual 'ExchangedTermGridConnection' because of 'ExchangedDataHouseholdToCompany'
Object Property 'termHasMeaningByContext' inferred on individual 'ContextDERHousehold' be

In [8]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

sentence = "I went to the bank to deposit some money."
tokenized_sentence = word_tokenize(sentence)
word = 'bank'

# Perform Word Sense Disambiguation for 'bank' in the given sentence
meaning = lesk(tokenized_sentence, word)
print(meaning, meaning.definition())

Synset('savings_bank.n.02') a container (usually with a slot in the top) for keeping money at home


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Define the SPARQL query
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oec: <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#>

    SELECT ?individual1 ?individual2
    WHERE {
        ?individual1 oec:termHasMeaningByContext ?individual2 .
    }
"""

try:
    # Execute SPARQL query
    results = onto.world.sparql(query)

    # Print results
    for row in results:
        individual1 = row[0]
        individual2 = row[1]
        print(f"Individual connected through 'termHasMeaningByContext': {individual1} to {individual2}")

except Exception as e:
    print("Error occurred while executing the SPARQL query:")
    print(e)

Individual connected through 'termHasMeaningByContext': C:\Users\Matheus\Documents\GitHub\Papers\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\ontology\imports\oec-extracted.ExchangedTermNetMetering to C:\Users\Matheus\Documents\GitHub\Papers\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\ontology\imports\oec-extracted.ContextDERHousehold
Individual connected through 'termHasMeaningByContext': C:\Users\Matheus\Documents\GitHub\Papers\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\ontology\imports\oec-extracted.ExchangedTermStorage to C:\Users\Matheus\Documents\GitHub\Papers\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\ontology\imports\oec-extracted.ContextDERHousehold
Individual connected through 'termHasMeaningByContext': C:\Users\Matheus\Documents\GitHub\Papers

In [10]:
# Print inferred information for each class, property, and individual
print("Inferred Classes:")
for cls in onto.classes():
    cls_name = cls.name.split('#')[-1] if '#' in cls.name else cls.name
    print(f"\nClass: {cls_name}")
    instances = list(cls.instances())
    if instances:
        instance_names = [inst.name.split('#')[-1] if '#' in inst.name else inst.name for inst in instances]
        print(f"Inferred Instances: {instance_names}")
    else:
        print("No inferred instances.")

print("\nInferred Object Properties:")
for prop in onto.object_properties():
    prop_name = prop.name.split('#')[-1] if '#' in prop.name else prop.name
    print(f"\nObject Property: {prop_name}")
    relations = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name,
                  obj.name.split('#')[-1] if '#' in obj.name else obj.name) for subj, obj in prop.get_relations()]
    if relations:
        for subj, obj in relations:
            print(f"Subject: {subj}, Object: {obj}")
    else:
        print("No inferred relations.")

print("\nInferred Data Properties:")
for data_prop in onto.data_properties():
    data_prop_name = data_prop.name.split('#')[-1] if '#' in data_prop.name else data_prop.name
    print(f"\nData Property: {data_prop_name}")
    values = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name, value) for subj, value in data_prop.get_relations()]
    if values:
        for subj, value in values:
            print(f"Subject: {subj}, Value: {value}")
    else:
        print("No inferred values.")

print("\nInferred Individuals:")
for ind in onto.individuals():
    ind_name = ind.name.split('#')[-1] if '#' in ind.name else ind.name
    print(f"\nIndividual: {ind_name}")
    types = [typ.name.split('#')[-1] if '#' in typ.name else typ.name for typ in ind.is_a]
    if types:
        print(f"Inferred Types: {types}")
    else:
        print("No inferred types.")

Inferred Classes:

Class: actor
Inferred Instances: ['ActorDERHousehold', 'ContextDERHousehold', 'ActorTypeDER', 'ActorEnergyCompany', 'ActorTypeCompany', 'ContextEnergyCompany']

Class: context
Inferred Instances: ['ActorDERHousehold', 'ContextDERHousehold', 'ContextUnknown']

Class: organization
Inferred Instances: ['ContextEnergyCompany']

Class: actorRole
Inferred Instances: ['ActorDERHousehold', 'ActorEnergyCompany']

Class: energySector
No inferred instances.

Class: sector
No inferred instances.

Class: actorType
Inferred Instances: ['ContextDERHousehold', 'ActorTypeDER', 'ActorTypeCompany']

Class: userAtentionFocus
No inferred instances.

Class: exchangedData
Inferred Instances: ['ExchangedDataHouseholdToCompany', 'ExchangedTermGridConnection', 'ExchangedTermNetMetering', 'ExchangedTermStorage']

Class: exchangedDataLog
No inferred instances.

Class: Receiver
Inferred Instances: ['ActorEnergyCompany']

Class: Sender
Inferred Instances: ['ActorDERHousehold']

Class: term
Inferr

In [11]:
# Load the ontology using its IRI
onto = get_ontology("http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted").load()

# Retrieve individuals starting with "ExchangedTerm"
matching_individuals = [ind for ind in onto.individuals() if ind.name.startswith("ExchangedTerm")]

# Define the object property to focus on
target_property = onto.termHasMeaningByContext

# Iterate through the filtered individuals to find those with the target object property
for term_individual in matching_individuals:
    # Check if the individual has the target object property
    if hasattr(term_individual, "termHasMeaningByContext"):
        related_property = term_individual.termHasMeaningByContext
        if related_property:
            print(f"Individual '{term_individual.name}' has the 'termHasMeaningByContext' object property.")
            for related_ind in related_property:
                print(f" - Related individual: '{related_ind.name}'")
        else:
            print(f"Individual '{term_individual.name}' has 'termHasMeaningByContext' but no related individuals.")


Individual 'ExchangedTermGridConnection' has 'termHasMeaningByContext' but no related individuals.
Individual 'ExchangedTermNetMetering' has the 'termHasMeaningByContext' object property.
 - Related individual: 'ContextDERHousehold'
Individual 'ExchangedTermStorage' has the 'termHasMeaningByContext' object property.
 - Related individual: 'ContextDERHousehold'


In [18]:
from owlready2 import *
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Load the ontology using its IRI
onto = get_ontology("http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted").load()

# Retrieve individuals starting with "ExchangedTerm"
matching_individuals = [ind for ind in onto.individuals() if ind.name.startswith("ExchangedTerm")]

# NLTK context with terms and meanings
nltk_context = {
    "ContextDERHousehold": {
        "Storage": "cells",
        # Define other terms and meanings within the ContextDERHousehold as needed
    }
}

# Iterate through the filtered individuals to find those with the target object property
for term_individual in matching_individuals:
    # Initialize variables for storing context and related terms
    context_meaning = ""
    connected_to = []

    # Check if the individual has the target object property
    if hasattr(term_individual, "termHasMeaningByContext"):
        related_property = term_individual.termHasMeaningByContext

        # Loop through related properties and extract names
        for related_ind in related_property:
            related_ind_name = related_ind.name if related_ind.name else ""
            connected_to.append(related_ind_name)

    # Check if termLexicon data property exists and has values
    if hasattr(term_individual, "termLexicon") and term_individual.termLexicon:
        term_lexicon = term_individual.termLexicon[0]  # Assuming there's a single value

        # Check if the termLexicon is found in ontology-defined contexts
        if term_lexicon in nltk_context.get("ContextDERHousehold", {}):
            context_meaning = nltk_context["ContextDERHousehold"][term_lexicon]

    # Print the information for the current individual
    print(f"Individual: '{term_individual.name}'")
    print(f" - Object Property: 'termHasMeaningByContext'")
    print(f" - Connects to: {', '.join(connected_to) if connected_to else 'None'}")
    if hasattr(term_individual, "termLexicon") and term_individual.termLexicon:
        print(f" - Term Lexicon: {term_lexicon}")
    if context_meaning:
        print(f" - Context Meaning: {context_meaning}")
    else:
        print(" - No matching context found in termHasMeaningByContext. NLTK context used.")


Individual: 'ExchangedTermGridConnection'
 - Object Property: 'termHasMeaningByContext'
 - Connects to: None
 - No matching context found in termHasMeaningByContext. NLTK context used.
Individual: 'ExchangedTermNetMetering'
 - Object Property: 'termHasMeaningByContext'
 - Connects to: ContextDERHousehold
 - Term Lexicon: bank
 - No matching context found in termHasMeaningByContext. NLTK context used.
Individual: 'ExchangedTermStorage'
 - Object Property: 'termHasMeaningByContext'
 - Connects to: ContextDERHousehold
 - Term Lexicon: Storage
 - Context Meaning: cells


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from owlready2 import *

# Load the ontology using its IRI
onto = get_ontology("http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted").load()

# Dictionary to store relationships
relationships = {}

# Iterate through all individuals
for ind in onto.individuals():
    relationships[ind] = {}
    for prop in ind.get_properties():
        # Check if the property is an object property
        if isinstance(prop, ObjectPropertyClass):
            # Check if the property has values for the individual
            if prop[ind]:
                # Add related individuals to the dictionary along with the property
                relationships[ind][prop.name] = prop[ind]

# Display individuals connected through object properties and the respective properties
for ind, connections in relationships.items():
    if connections:
        for prop, connected_inds in connections.items():
            for connected in connected_inds:
                print(f"Individual '{ind.name}' connected to '{connected.name}' through object property '{prop}'.")


Individual 'ActorDERHousehold' connected to 'ActorTypeDER' through object property 'actorHasType'.
Individual 'ActorDERHousehold' connected to 'ContextDERHousehold' through object property 'actorHasContext'.
Individual 'ActorDERHousehold' connected to 'ContextDERHousehold' through object property 'contextDefinedByActorType'.
Individual 'ExchangedDataHouseholdToCompany' connected to 'ActorEnergyCompany' through object property 'exchangedDataHasReceiver'.
Individual 'ExchangedDataHouseholdToCompany' connected to 'ActorDERHousehold' through object property 'exchangedDataHasSender'.
Individual 'ExchangedDataHouseholdToCompany' connected to 'ExchangedTermGridConnection' through object property 'exchangedDataHasTerm'.
Individual 'ExchangedDataHouseholdToCompany' connected to 'ExchangedTermNetMetering' through object property 'exchangedDataHasTerm'.
Individual 'ExchangedDataHouseholdToCompany' connected to 'ExchangedTermStorage' through object property 'exchangedDataHasTerm'.
Individual 'Exch

In [13]:
from nltk.corpus import wordnet

# Function to get meanings of words using WordNet
def get_meanings(word):
    meanings = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            meanings.append(lemma.name())
    return list(set(meanings))

# Sample text
text = "NLTK is a powerful library for natural language processing."

# Tokenize the text into words
words = text.split()

# Dictionary to store words and their meanings
words_meanings = {}

# Get meanings for each word
for word in words:
    meanings = get_meanings(word)
    words_meanings[word] = meanings

# Print words and their meanings
for word, meanings in words_meanings.items():
    print(f"Word: {word}")
    if meanings:
        print(f"Meanings: {', '.join(meanings)}")
    else:
        print("No meanings found")
    print()


Word: NLTK
No meanings found

Word: is
Meanings: live, make_up, represent, equal, comprise, cost, be, exist, personify, constitute, follow, embody

Word: a
Meanings: amp, a, type_A, A, axerophthol, vitamin_A, deoxyadenosine_monophosphate, ampere, angstrom_unit, angstrom, adenine, antiophthalmic_factor, group_A

Word: powerful
Meanings: hefty, muscular, mightily, right, potent, herculean, brawny, sinewy, powerful, mighty, knock-down

Word: library
Meanings: program_library, subroutine_library, depository_library, library

Word: for
No meanings found

Word: natural
Meanings: born, natural, instinctive, lifelike, rude, cancel, raw, innate

Word: language
Meanings: speech, language, speech_communication, voice_communication, words, spoken_communication, oral_communication, spoken_language, nomenclature, linguistic_process, terminology, linguistic_communication, lyric

Word: processing.
No meanings found

